In [1]:
#importing the module
from langchain_openai import OpenAIEmbeddings #this class we used to load openai embedding models
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate #this class we used to create dynamic or role based prompt
import os
from langchain_core.runnables import RunnableBranch,RunnableParallel,RunnablePassthrough,RunnableLambda,RunnableSequence
from langchain_openai import ChatOpenAI #this class we used to load openai llm models
from langchain_chroma import Chroma #this class we used to create chroma vector store database
from langchain_community.document_loaders import TextLoader,PDFPlumberLoader,DirectoryLoader
from langchain_core.output_parsers import StrOutputParser,PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from typing import TypedDict,Literal,Annotated
from pydantic import BaseModel,Field,computed_field

from dotenv import load_dotenv


In [2]:
from langchain_community.retrievers import WikipediaRetriever

In [3]:
#creating an object of WikipediaRetriever class
retriever = WikipediaRetriever(
    top_k_results=2,
    lang="en"
)
retriever

WikipediaRetriever(wiki_client=<module 'wikipedia' from 'c:\\anaconda\\envs\\hf-gpu\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=2)

# defining query

In [5]:
query = "the geopolitical history of india and pakistan from the perspective of a china"
query

'the geopolitical history of india and pakistan from the perspective of a china'

# retrieving the document from wikepedia website

In [6]:
#based on the query we r checking the information or document on wikepedia website
response = retriever.invoke(input=query)
response

[Document(metadata={'title': 'China–India relations', 'summary': "China and India maintained peaceful relations for thousands of years, but their relationship has varied since the Chinese Communist Party (CCP)'s victory in the Chinese Civil War in 1949 and the annexation of Tibet by the People's Republic of China. The two nations have sought economic cooperation with each other, while frequent border disputes and economic nationalism in both countries are major points of contention.\nCultural and economic relations between China and India date back to ancient times. The Silk Road not only served as a major trade route between India and China, but is also credited for facilitating the spread of Buddhism from India to East Asia. During the 19th century, China was involved in a growing opium trade with the East India Company, which exported opium grown in India. During World War II, both British India and the Republic of China (ROC) played a crucial role in halting the progress of Imperia

In [10]:
for i,doc in enumerate(response):
    print(f"\n--- Result {i+1} ---")
    print(f"content:\n{doc.page_content}")


--- Result 1 ---
content:
China and India maintained peaceful relations for thousands of years, but their relationship has varied since the Chinese Communist Party (CCP)'s victory in the Chinese Civil War in 1949 and the annexation of Tibet by the People's Republic of China. The two nations have sought economic cooperation with each other, while frequent border disputes and economic nationalism in both countries are major points of contention.
Cultural and economic relations between China and India date back to ancient times. The Silk Road not only served as a major trade route between India and China, but is also credited for facilitating the spread of Buddhism from India to East Asia. During the 19th century, China was involved in a growing opium trade with the East India Company, which exported opium grown in India. During World War II, both British India and the Republic of China (ROC) played a crucial role in halting the progress of Imperial Japan. After India became independent 

# vector store retrievers

In [11]:
load_dotenv()


#creating an object of embedding models
emb_models = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=128
)


#creating and object llm chatmodel 
chat_model = ChatOpenAI(
    temperature=0 #we called as creative parameter
)


In [12]:
# Step 1: Your source documents
from langchain.schema import Document
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]
documents

[Document(metadata={}, page_content='LangChain helps developers build LLM applications easily.'),
 Document(metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'),
 Document(metadata={}, page_content='Embeddings convert text into high-dimensional vectors.'),
 Document(metadata={}, page_content='OpenAI provides powerful embedding models.')]

In [16]:
#creating a chroma vector store database 
from langchain_chroma import Chroma #this class we used to create chroma vector store database
vector_store = Chroma(
    collection_name="raees",
    embedding_function=emb_models,
    persist_directory="sample_chromaDB",
    
)
vector_store

# if i want to add document in vector store DB

In [18]:
vector_store.add_documents(documents=documents)

['2c7443ab-ba87-408e-93cd-b83d6b7bd2cc',
 'a8574e38-bc4f-4b1f-9500-0196c4786244',
 'e55bdb75-31d5-4576-8155-7c565299651f',
 '7c5afb8c-4be0-4480-a5c1-92f918a6b3c2']

In [19]:
#to show the embedding vector is store in vectore storeDB or not.
vector_store.get(include=["metadatas", "documents"])


{'ids': ['2c7443ab-ba87-408e-93cd-b83d6b7bd2cc',
  'a8574e38-bc4f-4b1f-9500-0196c4786244',
  'e55bdb75-31d5-4576-8155-7c565299651f',
  '7c5afb8c-4be0-4480-a5c1-92f918a6b3c2'],
 'embeddings': None,
 'documents': ['LangChain helps developers build LLM applications easily.',
  'Chroma is a vector database optimized for LLM-based search.',
  'Embeddings convert text into high-dimensional vectors.',
  'OpenAI provides powerful embedding models.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None, None, None, None]}

# Convert vectorstore into a retriever

In [23]:
# Convert vectorstore into a retriever
retriever2 = vector_store.as_retriever(search_kwargs={'k': 2},search_type="similarity")
retriever2

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000018250BFE620>, search_kwargs={'k': 2})

In [24]:
#defining query from user
query2 = "What is Chroma used for?"

#searching the query on vector store DB
result = retriever2.invoke(query2)
result

[Document(id='a8574e38-bc4f-4b1f-9500-0196c4786244', metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'),
 Document(id='2c7443ab-ba87-408e-93cd-b83d6b7bd2cc', metadata={}, page_content='LangChain helps developers build LLM applications easily.')]

In [25]:
for i,doc in enumerate(result):
    print(f"\n--- Result {i+1} ---")
    print(f"content:\n{doc.page_content}")


--- Result 1 ---
content:
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
content:
LangChain helps developers build LLM applications easily.


In [ ]:
vector_store.similarity_search_with_score(query2, k=3) #if i want to show similarity score too then we have to used this method

[(Document(id='a8574e38-bc4f-4b1f-9500-0196c4786244', metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'),
  0.6989080309867859),
 (Document(id='2c7443ab-ba87-408e-93cd-b83d6b7bd2cc', metadata={}, page_content='LangChain helps developers build LLM applications easily.'),
  1.4694808721542358),
 (Document(id='e55bdb75-31d5-4576-8155-7c565299651f', metadata={}, page_content='Embeddings convert text into high-dimensional vectors.'),
  1.8060450553894043)]

In [ ]:
'''
vector_store.similarity_search_with_score(query2, k=3) 
agar above syntax used karta hu toh woh query ko vector database mei search karke result degha lekin algorithm jo used kar 
raha hai woh hai COSINE SIMILARITY

agar koi searching technique karne ho toh retrievers object used karna hai jismei hum algorithm define kar sakte hai
like MMR(maxial marginal relevance)
'''

'\nvector_store.similarity_search_with_score(query2, k=3) \nagar above syntax used karta hu toh woh query ko vector database mei search karke result degha lekin algorithm jo used kar \nraha hai woh hai COSINE SIMILARITY\n\nagar koi searching technique karne ho toh retrievers object used karna hai jismei hum algorithm define kar sakte hai\nlike MMR\n'

# MMR(maxial marginal relevance) kab used karna hai jab vector database mei duplicat vector bohat ho avoid karne ke liye

In [29]:
from langchain_community.vectorstores import FAISS

In [32]:
# Sample documents
docs1 = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]
docs1

[Document(metadata={}, page_content='LangChain makes it easy to work with LLMs.'),
 Document(metadata={}, page_content='LangChain is used to build LLM based applications.'),
 Document(metadata={}, page_content='Chroma is used to store and search document embeddings.'),
 Document(metadata={}, page_content='Embeddings are vector representations of text.'),
 Document(metadata={}, page_content='MMR helps you get diverse results when doing similarity search.'),
 Document(metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.')]

In [36]:
#creating an object of FAISS vectore store database
vector_db_faiss = FAISS.from_documents(
    embedding=emb_models,
    documents=docs1,
    
)
vector_db_faiss

# converting faiss vector store database to retrievers

In [41]:
retriever3 = vector_db_faiss.as_retriever(
    # Retrieve more documents with higher diversity
    # Useful if your dataset has many similar documents
   
        search_type="mmr",
        search_kwargs={'k': 6, 'lambda_mult': 0.5} #lambda_mult varies from 0 to 1 agar 1 hua behave cosine similarity ki tarah
        #0 means bohat diverse result i am getting
    )
retriever3

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000182518EFBB0>, search_type='mmr', search_kwargs={'k': 6, 'lambda_mult': 0.5})

In [44]:
#defining query from user
query3 = "What is Langchain?"

In [45]:
solution = retriever3.invoke(input=query3)
solution

[Document(id='4af58273-521f-48ea-a859-32ef8a85ba46', metadata={}, page_content='LangChain is used to build LLM based applications.'),
 Document(id='a4eb6327-230d-4169-a07d-7cf1e7b5a349', metadata={}, page_content='Chroma is used to store and search document embeddings.'),
 Document(id='7426aa6d-7097-482d-8c06-b9ccbabef5b8', metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.'),
 Document(id='745511e8-65b8-4e55-b31b-47500a0e2051', metadata={}, page_content='LangChain makes it easy to work with LLMs.'),
 Document(id='791d1eda-d641-4e5e-b8de-e125d0763b20', metadata={}, page_content='Embeddings are vector representations of text.'),
 Document(id='7e94ae86-9568-454f-bdd7-54137767112d', metadata={}, page_content='MMR helps you get diverse results when doing similarity search.')]

# multi query Retrieval kab used karna hai:- jab user ke input query mei ambuigity or multiple meaning ho sakti hai or incorrect query ho sakti tab we have to used

In [46]:
from langchain.retrievers import MultiQueryRetriever

In [47]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]
all_docs

[Document(metadata={'source': 'H1'}, page_content='Regular walking boosts heart health and can reduce symptoms of depression.'),
 Document(metadata={'source': 'H2'}, page_content='Consuming leafy greens and fruits helps detox the body and improve longevity.'),
 Document(metadata={'source': 'H3'}, page_content='Deep sleep is crucial for cellular repair and emotional regulation.'),
 Document(metadata={'source': 'H4'}, page_content='Mindfulness and controlled breathing lower cortisol and improve mental clarity.'),
 Document(metadata={'source': 'H5'}, page_content='Drinking sufficient water throughout the day helps maintain metabolism and energy.'),
 Document(metadata={'source': 'I1'}, page_content='The solar energy system in modern homes helps balance electricity demand.'),
 Document(metadata={'source': 'I2'}, page_content='Python balances readability with power, making it a popular system design language.'),
 Document(metadata={'source': 'I3'}, page_content='Photosynthesis enables plants

In [48]:
# Create FAISS vector store
vectorstorefaiss = FAISS.from_documents(documents=all_docs, embedding=emb_models)
vectorstorefaiss

In [49]:
# converting vectorstorefaiss  database object to retriever using MultiQueryRetriever class
multi_retrivers = MultiQueryRetriever.from_llm(retriever=vectorstorefaiss.as_retriever(search_type="mmr",
        search_kwargs={'k': 2, 'lambda_mult': 0.25}),
                                          llm=chat_model
                                          )
multi_retrivers

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000182518F33D0>, search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.25}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is\n    to generate 3 different versions of the given user\n    question to retrieve relevant documents from a vector  database.\n    By generating multiple perspectives on the user question,\n    your goal is to help the user overcome some of the limitations\n    of distance-based similarity search. Provide these alternative\n    questions separated by newlines. Original question: {question}')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001824DE4A620>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001824

In [50]:
# Query
query = "How to improve energy levels and maintain balance?"
query

'How to improve energy levels and maintain balance?'

In [51]:
multiquery_results= multi_retrivers.invoke(query)
for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.


# ContextualCompressionRetriever kab used karna hai user query ke hisab se response dikhana hai applying contextual compression

In [52]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [53]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]
docs

[Document(metadata={'source': 'Doc1'}, page_content='The Grand Canyon is one of the most visited natural wonders in the world.\n        Photosynthesis is the process by which green plants convert sunlight into energy.\n        Millions of tourists travel to see it every year. The rocks date back millions of years.'),
 Document(metadata={'source': 'Doc2'}, page_content='In medieval Europe, castles were built primarily for defense.\n        The chlorophyll in plant cells captures sunlight during photosynthesis.\n        Knights wore armor made of metal. Siege weapons were often used to breach castle walls.'),
 Document(metadata={'source': 'Doc3'}, page_content='Basketball was invented by Dr. James Naismith in the late 19th century.\n        It was originally played with a soccer ball and peach baskets. NBA is now a global league.'),
 Document(metadata={'source': 'Doc4'}, page_content='The history of cinema began in the late 1800s. Silent films were the earliest form.\n        Thomas Edis

# # Create a FAISS vector DB to store from the documents

In [54]:
# Create a FAISS vector store from the documents
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

# converting vector store db object to retrievrs

In [55]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5,'lambda_mult': 0.25},search_type="mmr")
base_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018286A606A0>, search_type='mmr', search_kwargs={'k': 5, 'lambda_mult': 0.25})

# # Set up the compressor using an LLM

In [56]:
# Set up the compressor using an LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")
compressor = LLMChainExtractor.from_llm(llm)

# Create the contextual compression retriever

In [57]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)
compression_retriever

ContextualCompressionRetriever(base_compressor=LLMChainExtractor(llm_chain=PromptTemplate(input_variables=['context', 'question'], input_types={}, output_parser=NoOutputParser(), partial_variables={}, template='Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT.\n\nRemember, *DO NOT* edit the extracted parts of the context.\n\n> Question: {question}\n> Context:\n>>>\n{context}\n>>>\nExtracted relevant parts:')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000018286A60FA0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000018286A61C00>, root_client=<openai.OpenAI object at 0x0000018286A63D30>, root_async_client=<openai.AsyncOpenAI object at 0x0000018286A61120>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| NoOutputParser(), get_input=<function default_get_input at

In [58]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)
compressed_results

[Document(metadata={'source': 'Doc1'}, page_content='Photosynthesis is the process by which green plants convert sunlight into energy.'),
 Document(metadata={'source': 'Doc2'}, page_content='The chlorophyll in plant cells captures sunlight during photosynthesis.')]

In [59]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
